# Linkedin Scraping

https://medium.com/@gerrysabar/scraping-linkedin-profile-using-python-selenium-88cb64888cf
https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/

In [2]:
# import web driver
from selenium import webdriver
import pandas as pd
import numpy as np
import re 
import csv
import time 
import random
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens') #Bert pretrained model 
from scipy import spatial

In [30]:
class Frame():  
    
    def __init__(self,t):
        self.df = pd.DataFrame(columns = ['search key', 'name', 'job title', 'school','location','linkedin url'])
        self.m = pd.read_excel("/Users/guapofish/Desktop/500 for matching.xlsx")
        self.t = t 
    
    def search_Keywrds(self,Name):
        #writer = csv.writer(open('testing.csv', 'w')) # preparing csv file to store parsing result later
        #writer.writerow(['name', 'job_title', 'schools', 'location', 'ln_url'])

        # specifies the path to the chromedriver.exe
        driver = webdriver.Chrome('/Users/guapofish/Desktop/Linkedin Scraping/chromedriver')

        # driver.get method() will navigate to a page given by the URL address  
        driver.get('https://www.linkedin.com/')
        time.sleep(random.randint(1, 10)/3)
        driver.find_element_by_xpath('//a[text()="Sign in"]').click()

        username_input = driver.find_element_by_name('session_key')
        username_input.send_keys('qihangdavid@gmail.com')

        password_input = driver.find_element_by_name('session_password')
        password_input.send_keys('13917933131yutou')

        # click on the sign in button
        # we're finding Sign in text button as it seems this element is seldom to be changed
        driver.find_element_by_xpath('//button[text()="Sign in"]').click()

        driver.get('https://www.google.com/')

        search_input = driver.find_element_by_name('q')
        # let google find any linkedin user with keyword "python developer" and "San Francisco"
        search_input.send_keys('site:linkedin.com/in/ {0}'.format(Name))
        search_input.send_keys(Keys.RETURN)

        # grab all linkedin profiles from first page at Google
        profiles = driver.find_elements_by_xpath('//*[@class="r"]/a[1]')
        profiles = [profile.get_attribute('href') for profile in profiles][:self.t]

        # visit each profile in linkedin and grab detail we want to get

        l_name = []
        l_job_title = []
        l_schools = []
        l_location = []
        l_ln_url = []
        #l_keywrds = [title for i in range(len(profiles))]   

        for profile in profiles:
            driver.get(profile)

            try:
                sel = Selector(text=driver.page_source)
                name = sel.xpath('//title/text()').extract_first().split(' | ')[0]
                name = name[re.search(" ", name).end():] #delete the number before the blank 
                jobtitle = sel.xpath('//h2/text()').extract()[2].strip() #sel.xpath('//h2/text()').extract_first().strip()
                schools = ', '.join(sel.xpath('//*[contains(@class, "pv-entity__school-name")]/text()').extract())
                location = sel.xpath('//*[@class="t-16 t-black t-normal inline-block"]/text()').extract_first().strip()
                ln_url = driver.current_url
                """
                you can add another logic in case parsing is failed, ie because no job title is found
                because the linkedin user isn't add it
                """
            except:
                print('failed')

            # print to console for testing purpose

            l_name.append(name)
            l_job_title.append(jobtitle)
            l_schools.append(schools)
            l_location.append(location)
            l_ln_url.append(ln_url)
            
            '''
            print('\n')
            print(name)        
            print(job_title)
            print(schools)
            print(location)
            print(ln_url)
            print('\n')
            
            '''

        d = {'name':l_name , 'job title': l_job_title, 'school':l_schools ,'location':l_location,
             'linkedin url': l_ln_url}
        f = pd.DataFrame(data=d)
        #self.df = pd.concat([self.df, f], axis = 0).reset_index(drop = True)
        #writer.writerow([name, job_title, schools, location, ln_url])      

        return f
    
    def concat(self,Name):
        self.df = pd.concat([self.df,self.search_Keywrds(self.t,Name)], axis = 0)
        return self.df.reset_index(drop = True)
    
    def get_match_table(self): 
        #clean the NaN value, combine First name and last name etc
        self.m['name'] = (self.m[['First Name', 'Last Name']]).agg(' '.join, axis=1) 
        self.m['Job Title'] = self.m['Job Title'].replace(np.NaN, 'None', regex=True)
        self.m['Company Name'] = self.m['Company Name'].replace(np.NaN, 'None', regex=True)
        self.m['job title'] = self.m[['Job Title', 'Company Name']].agg(' '.join, axis=1) 
        self.m = self.m[['Backstop Party ID','name','job title','City']]
        self.m = self.m.rename(columns={"City": "location"})
        return self.m

    def cosine_similarity(self,sentense1,sentense2):
        dataSetI = model.encode(sentense1)
        dataSetII = model.encode(sentense2)
        result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
        return result

    def get_score(self,df0,l):  
        #model = SentenceTransformer('bert-base-nli-mean-tokens') #Bert pretrained model 
        items = ['name','job title','location']
        w = np.array([0.60,0.30,0.1]) #Name:60%, job title:30%, location:10%
        score = []
        for j in range(self.t):
            x = df0[items].iloc[j]  
            s0 = self.cosine_similarity(x[items[0]],l[0])
            s1 = self.cosine_similarity(x[items[1]],l[1])
            s2 = self.cosine_similarity(x[items[2]],l[2])
            #print(s0,s1,s2)
            score.append(np.sum(np.array([s0,s1,s2])*w))
            #print(score)

        df0['score'] = score
        idx = df0.score.argmax()
        df0['warning level'] = idx
        df0 = df0.rename(columns={"name":"Name","job title":"Job Title","location": "Location"})
        return df0 #.loc[idx],df0.loc[idx].score,idx

    def get_score_df(self,test): 
        appended_data = []
        for i in range(len(test)):
            n = test['name'].iloc[i]
            job = test['job title'].iloc[i]
            city = test['location'].iloc[i]
            df0 = self.search_Keywrds(n)
            l = [n,job,city]        
            data = self.get_score(df0,l)
            appended_data.append(data)
        appended_data = pd.concat(appended_data, axis = 1)

        return appended_data.transpose()

    def get_final_df(self,start,end):
        dff = self.get_match_table()[start:end]
        scape_df = self.get_score_df(dff)
        a = scape_df.reset_index(drop = True)
        b = dff.reset_index(drop = True)
        return pd.concat([b,a], axis = 1)



In [34]:
a = Frame(5)
matchTable = a.get_match_table()
matchTable[92:95]


,Backstop Party ID,name,job title,location
92,298250459,Tarek Rizk,Vice President Alternative Investments Funds a...,New York
93,298250461,Phillip Schaeffer,Fund Manager Scott's Cove Management LLC,New York
94,298250463,Cathlin Sheehan,Associate Deutsche Bank,Newark


In [24]:
a = Frame(3)
a.get_final_df(92,95)

,Backstop Party ID,name,job title,location,Name,Job Title,school,Location,linkedin url,score,warning level
0,298250459,Tarek Rizk,Vice President Alternative Investments Funds a...,New York,Tarek Rizk,Highlights,"Harvard Business School, University of Michiga...",United States,https://www.linkedin.com/in/tarek-rizk-02678318/,0.623398,1
1,298250461,Phillip Schaeffer,Fund Manager Scott's Cove Management LLC,New York,Phillip Schaeffer,Highlights,"Harvard Business School, University of Virginia","New York, New York",https://www.linkedin.com/in/phillip-schaeffer-...,0.717117,0
2,298250463,Cathlin Sheehan,Associate Deutsche Bank,Newark,Jen Williams,Highlights,"Saint Michael's College, The University of Auc...","Hamden, Connecticut",https://www.linkedin.com/in/jenbutsonwilliams/,0.479707,0


In [25]:
a = Frame(5)
a.get_final_df(92,95)

,Backstop Party ID,name,job title,location,Name,Job Title,school,Location,linkedin url,score,warning level
0,298250459,Tarek Rizk,Vice President Alternative Investments Funds a...,New York,Tarek Rizk,Highlights,"Harvard Business School, University of Michiga...",United States,https://www.linkedin.com/in/tarek-rizk-02678318/,0.623398,1
1,298250461,Phillip Schaeffer,Fund Manager Scott's Cove Management LLC,New York,Phillip Schaeffer,Highlights,"Harvard Business School, University of Virginia","New York, New York",https://www.linkedin.com/in/phillip-schaeffer-...,0.717117,0
2,298250463,Cathlin Sheehan,Associate Deutsche Bank,Newark,Jen Williams,Highlights,"Saint Michael's College, The University of Auc...","Hamden, Connecticut",https://www.linkedin.com/in/jenbutsonwilliams/,0.479707,0


In [28]:
bb = Frame(3)
bb.get_final_df(92,95)

,Backstop Party ID,name,job title,location,Name,Job Title,school,Location,linkedin url,score,warning level
0,298250459,Tarek Rizk,Vice President Alternative Investments Funds a...,New York,Tarek Rizk,Global Head of Alternative Investments,"Harvard Business School, University of Michiga...",United States,https://www.linkedin.com/in/tarek-rizk-02678318/,0.839668,1
1,298250461,Phillip Schaeffer,Fund Manager Scott's Cove Management LLC,New York,Phillip Schaeffer,"Managing Member, Scott's Cove Management LLC","Harvard Business School, University of Virginia","New York, New York",https://www.linkedin.com/in/phillip-schaeffer-...,0.950165,0
2,298250463,Cathlin Sheehan,Associate Deutsche Bank,Newark,Jen Williams,Marketing & Communications Consultant,"Saint Michael's College, The University of Auc...","Hamden, Connecticut",https://www.linkedin.com/in/jenbutsonwilliams/,0.615675,0


In [29]:
b = Frame(5)
b.get_final_df(92,95)

,Backstop Party ID,name,job title,location,Name,Job Title,school,Location,linkedin url,score,warning level
0,298250459,Tarek Rizk,Vice President Alternative Investments Funds a...,New York,Tarek Rizk,Global Head of Alternative Investments,"Harvard Business School, University of Michiga...",United States,https://www.linkedin.com/in/tarek-rizk-02678318/,0.839668,1
1,298250461,Phillip Schaeffer,Fund Manager Scott's Cove Management LLC,New York,Phillip Schaeffer,"Managing Member, Scott's Cove Management LLC","Harvard Business School, University of Virginia","New York, New York",https://www.linkedin.com/in/phillip-schaeffer-...,0.950165,0
2,298250463,Cathlin Sheehan,Associate Deutsche Bank,Newark,Jen Williams,Marketing & Communications Consultant,"Saint Michael's College, The University of Auc...","Hamden, Connecticut",https://www.linkedin.com/in/jenbutsonwilliams/,0.615675,0


# Appendix

In [47]:
'''

def get_match_table(match): 
    #clean the NaN value, combine First name and last name etc
    match['name'] = match[['First Name', 'Last Name']].agg(' '.join, axis=1) 
    match['Job Title'] = match['Job Title'].replace(np.NaN, 'None', regex=True)
    match['Company Name'] = match['Company Name'].replace(np.NaN, 'None', regex=True)
    match['job title'] = match[['Job Title', 'Company Name']].agg(' '.join, axis=1) 
    match = match[['Backstop Party ID','name','job title','City']]
    match = match.rename(columns={"City": "location"})
    return match

def cosine_similarity(sentense1,sentense2):
    dataSetI = model.encode(sentense1)
    dataSetII = model.encode(sentense2)
    result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
    return result

def get_score(df0,l):  
    #model = SentenceTransformer('bert-base-nli-mean-tokens') #Bert pretrained model 
    items = ['name','job title','location']
    w = np.array([0.45,0.45,0.1]) #Name:45%, job title:45%, location:10%
    score = []
    for j in range(3):
        x = df0[items].iloc[j]  
        s0 = cosine_similarity(x[items[0]],l[0])
        s1 = cosine_similarity(x[items[1]],l[1])
        s2 = cosine_similarity(x[items[2]],l[2])
        #print(s0,s1,s2)
        score.append(np.sum(np.array([s0,s1,s2])*w))
        #print(score)
    
    df0['score'] = score
    idx = df0.score.argmax()
    df0['warning level'] = idx
    df0 = df0.rename(columns={"name":"Name","job title":"Job Title","location": "Location"})
    return df0.loc[idx] #.loc[idx],df0.loc[idx].score,idx

def get_score_df(test,t): 
    appended_data = []
    for i in range(len(test)):
        n = test['name'].iloc[i]
        job = test['job title'].iloc[i]
        city = test['location'].iloc[i]
        df0 = a.search_Keywrds(n,t)
        l = [n,job,city]        
        data = get_score(df0,l)
        appended_data.append(data)
    appended_data = pd.concat(appended_data, axis = 1)
         
    return appended_data.transpose()

def get_final_df(dff,t):  
    scape_df = get_score_df(dff,t)
    a = scape_df.reset_index(drop = True)
    b = dff.reset_index(drop = True)
    return pd.concat([b,a], axis = 1)
'''

'\n\ndef get_match_table(match): \n    #clean the NaN value, combine First name and last name etc\n    match[\'name\'] = match[[\'First Name\', \'Last Name\']].agg(\' \'.join, axis=1) \n    match[\'Job Title\'] = match[\'Job Title\'].replace(np.NaN, \'None\', regex=True)\n    match[\'Company Name\'] = match[\'Company Name\'].replace(np.NaN, \'None\', regex=True)\n    match[\'job title\'] = match[[\'Job Title\', \'Company Name\']].agg(\' \'.join, axis=1) \n    match = match[[\'Backstop Party ID\',\'name\',\'job title\',\'City\']]\n    match = match.rename(columns={"City": "location"})\n    return match\n\ndef cosine_similarity(sentense1,sentense2):\n    dataSetI = model.encode(sentense1)\n    dataSetII = model.encode(sentense2)\n    result = 1 - spatial.distance.cosine(dataSetI, dataSetII)\n    return result\n\ndef get_score(df0,l):  \n    #model = SentenceTransformer(\'bert-base-nli-mean-tokens\') #Bert pretrained model \n    items = [\'name\',\'job title\',\'location\']\n    w = np.ar

In [44]:
test = matchTable[92:95]
n = test['name'].iloc[0]
job = test['job title'].iloc[0]
city = test['location'].iloc[0]
l = [n,job,city] 
get_score(df_10_20,l)

,Name,Job Title,school,Location,linkedin url,score,warning level
0,Tarek Rizk,Looking for new opportunity in seismic data pr...,"Management courses, Faculty Of Science",Other,https://www.linkedin.com/in/tarek-rizk-b223a627/,0.703183,1
1,Tarek Rizk,Marketing And Public Relations Specialist at W...,,Greater New York City Area,https://www.linkedin.com/in/tarek-rizk-18283b1a2/,0.810873,1
2,Tarek Rizk,Senior Vice President,"American University, Norwin High School",Washington D.C. Metro Area,https://www.linkedin.com/in/tarekrizk/,0.764884,1


In [24]:
pd.read_excel("/Users/guapofish/Desktop/500 for matching.xlsx")

,Contact Type,Backstop Party ID,First Name,Last Name,Job Title,Department,Company Name,City,State,Country
0,Person,298249317,Eileen,Casey,Managing Director,AIG Global Investments,AIG Asset Management,New York,NY,USA
1,Person,298249361,Susan,Holland,Commercial Property Manager,NaN,UCR Asset Services,Boston,MA,USA
2,Person,298249369,David,Mulholland,"Vice President, Investments",NaN,Arch Capital,Boston,MA,USA
3,Person,298249409,Guilherme,Del Bem,Asset Allocation - Credit Suisse Asset Management,Hedging-Griffo,Credit Suisse AG,New York,NY,USA
4,Person,298249423,Livia,Lima,Strategy and Operations Manager- LATAM,NaN,Waze,Mountain View,California,USA
...,...,...,...,...,...,...,...,...,...,...
495,Person,298251529,Virginia,Coccoli,Managing Director,NaN,Proctor Investments,New York,NY,USA
496,Person,298251531,Chas,Cocke,"Managing Director, Public Equities and Alterna...",NaN,Investure,Charlottesville,VA,USA
497,Person,298251533,Julie,Codina,Investment Analyst,NaN,Dimension Capital Management,Miami,FL,USA
498,Person,298251537,Jennifer,Coffey,CIO,NaN,Bright Capital Advisors,Woodside,CA,USA
